In [1]:
# Import required libraries
import sys
sys.path.insert(0, "/anaconda/lib/python2.7/site-packages")
import os
import numpy as np
import pandas as pd
from pandas import Series
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from __future__ import division
from scipy import stats
import seaborn as sns
os.chdir('/Users/wandawang/Desktop')
import re
from tpot import TPOTClassifier
from sklearn.cross_validation import train_test_split
import math

In [2]:
train = pd.read_csv('data/train_users_2.csv')
test = pd.read_csv('data/test_users.csv')
countries = pd.read_csv('data/countries.csv')
user_demo = pd.read_csv('data/age_gender_bkts.csv')
sessions = pd.read_csv('data/sessions.csv')

In [3]:
train.shape #213451.00,16
train.head(3)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US


In [4]:
train.dtypes 

id                          object
date_account_created        object
timestamp_first_active       int64
date_first_booking          object
gender                      object
age                        float64
signup_method               object
signup_flow                  int64
language                    object
affiliate_channel           object
affiliate_provider          object
first_affiliate_tracked     object
signup_app                  object
first_device_type           object
first_browser               object
country_destination         object
dtype: object

In [5]:
countries.head(3)
#countries.shape #(10, 7)

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,AU,-26.853388,133.275160,15297.7440,7741220.0,eng,0.00
1,CA,62.393303,-96.818146,2828.1333,9984670.0,eng,0.00
2,DE,51.165707,10.452764,7879.5680,357022.0,deu,72.61


In [6]:
#create age_buckets for training, to match with user_demo age buckets

#merge user_demo with countries?

In [7]:
user_demo.head(3) #age_bucket
#user_demo.shape #(420, 5)
#user_demo['age_bucket'].unique()

,age_bucket,country_destination,gender,population_in_thousands,year
0,100+,AU,male,1.0,2015.0
1,95-99,AU,male,9.0,2015.0
2,90-94,AU,male,47.0,2015.0


In [8]:
#sessions grouping by user
#Group by user_id, aggregate by number of counts (counting device_type as it is never NA), 
#and total sum of elapsed time in seconds
group_sessions = sessions.groupby("user_id").agg({'device_type':'count', 'secs_elapsed':'sum'})
group_sessions.columns = ['sum_secs_elapsed', 'counts']
group_sessions.reset_index(level=0, inplace=True)
group_sessions.head() 

,user_id,sum_secs_elapsed,counts
0,00023iyk9l,867896.0,40
1,0010k6l0om,586543.0,63
2,001wyh0pz8,282965.0,90
3,0028jgx1x1,297010.0,31
4,002qnbzfs5,6487080.0,789


In [9]:
sessions.head(3) #merge sessions with train... action

group_sessions.head(3) 

,user_id,sum_secs_elapsed,counts
0,00023iyk9l,867896.0,40
1,0010k6l0om,586543.0,63
2,001wyh0pz8,282965.0,90


In [10]:
#bucket all ages into format that user_demo is in for age
def agebuckets(ages):
    ageless =  [i for i in range(5,101,5)] # 5, 10, 15, 20...95, 100
    buckets = ['%d-%d' %(i, i+4) for i in range(0,100,5)] # 0-4, 5-9, 10-14...90-94, 95-99
    newlist = []
    for i in range(len(ages)):
        if math.isnan(ages[i]):
            newlist.append('NA')
        elif ages[i] <ageless[0]:
            newlist.append(buckets[0])
        elif ages[i] < ageless[1]:
            newlist.append(buckets[1])
        elif ages[i] < ageless[2]:
            newlist.append(buckets[2])
        elif ages[i] < ageless[3]:
            newlist.append(buckets[3])
        elif ages[i] < ageless[4]:
            newlist.append(buckets[4])
        elif ages[i] < ageless[5]:
            newlist.append(buckets[5])
        elif ages[i] < ageless[6]:
            newlist.append(buckets[6])
        elif ages[i] < ageless[7]:
            newlist.append(buckets[7])
        elif ages[i] < ageless[8]:
            newlist.append(buckets[8])
        elif ages[i] < ageless[9]:
            newlist.append(buckets[9])
        elif ages[i] < ageless[10]:
            newlist.append(buckets[10])
        elif ages[i] < ageless[11]:
            newlist.append(buckets[11])
        elif ages[i] < ageless[12]:
            newlist.append(buckets[12]) 
        elif ages[i] < ageless[13]:
            newlist.append(buckets[13]) 
        elif ages[i] < ageless[14]:
            newlist.append(buckets[14])
        elif ages[i] < ageless[15]:
            newlist.append(buckets[15])
        elif ages[i] < ageless[16]:
            newlist.append(buckets[16])
        elif ages[i] < ageless[17]:
            newlist.append(buckets[17])
        elif ages[i] < ageless[18]:
            newlist.append(buckets[18])
        elif ages[i] < ageless[19]:
            newlist.append(buckets[19]) 
        else:
            newlist.append('100+')
    return newlist

In [11]:
train.age = agebuckets(train.age)

In [12]:
train.age.unique() #train.age

array(['NA', '35-39', '55-59', '40-44', '45-49', '50-54', '30-34', '25-29',
       '15-19', '65-69', '60-64', '5-9', '70-74', '100+', '95-99', '20-24',
       '90-94', '75-79', '0-4', '85-89', '80-84'], dtype=object)

In [13]:
sessions.head(3)
#pd.isnull(sessions).any()  #user_id,action, secs_elapsed

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0


#country_destination is response variable

In [14]:
#train.rename(columns={'country_destination': 'class'}, inplace=True)
#train.dtypes 

In [15]:
#train_destination_class = train.iloc[:,-1]  #12
train_destination = train.iloc[:,-1]
# we store the class labels, which we need to predict, in a separate variable.
#train_destination_class.shape

In [16]:
#changing into datetime
train.date_account_created = pd.to_datetime(train.date_account_created)
train.timestamp_first_active = pd.to_datetime(train.timestamp_first_active, format = "%Y%m%d%H%M%S")
train.date_first_booking = pd.to_datetime(train.date_first_booking)
test.timestamp_first_active = pd.to_datetime(test.timestamp_first_active, format = "%Y%m%d%H%M%S")
test.date_account_created = pd.to_datetime(test.date_account_created)

In [17]:
#Feature engineering: time variables
train.head(3)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,NaT,-unknown-,NA,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,NaT,MALE,35-39,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,2010-08-02,FEMALE,55-59,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US


In [18]:
print "Dimensions", train.shape
print "Missingness for date_first_booking", sum(pd.isnull(train.date_first_booking))
print "Missingness for date_account_created", sum(pd.isnull(train.date_account_created))
print "Missingness for timestamp_first_active", sum(pd.isnull(train.timestamp_first_active))
print "Missingness for gender", sum(np.logical_or(train.gender == '-unknown-', train.gender == 'OTHER'))
print "Missingness for age", sum(pd.isnull(train.age))

Dimensions (213451, 16)
Missingness for date_first_booking 124543
Missingness for date_account_created 0
Missingness for timestamp_first_active 0
Missingness for gender 95970
Missingness for age 0


In [19]:
# date_account_created       datetime64[ns]
# timestamp_first_active     datetime64[ns]
# date_first_booking         datetime64[ns]


def timedif(L1, L2):
    timediflist = []
    for i in range(len(L1)):
        try:
            if (L1[i]-L2[i]).days <= -1:#datetime.timedelta(days=0):
                timediflist.append('before')
            elif (L1[i]-L2[i]).days ==0: #datetime.timedelta(days=1):
                timediflist.append('same day')
            else:
                timediflist.append('greater 1 day')
        except:
            timediflist.append('NA')
            
    return timediflist

In [20]:
np.unique(timedif(train.date_first_booking, train.date_account_created)) 
np.unique(timedif(train.date_first_booking, train.timestamp_first_active)) 

array(['NA', 'before', 'greater 1 day', 'same day'], 
      dtype='|S13')

In [21]:
#adding time lag columns
train['lag_account_created'] = timedif(train.date_first_booking, train.date_account_created)
train['lag_first_active'] = timedif(train.date_first_booking, train.timestamp_first_active)
train['lag_account_created_first_active'] = timedif(train.date_account_created, train.timestamp_first_active)

In [22]:
for cat in ['affiliate_provider', 'signup_app', 'first_affiliate_tracked', 'language', 'affiliate_channel']:
    print("Number of levels in category '{0}': \b {1:2.2f} ".format(cat, train[cat].unique().size))

Number of levels in category 'affiliate_provider':  18.00 
Number of levels in category 'signup_app':  4.00 
Number of levels in category 'first_affiliate_tracked':  8.00 
Number of levels in category 'language':  25.00 
Number of levels in category 'affiliate_channel':  8.00 


In [23]:
#Missigness
pd.isnull(train).any() #date_first_booking,age,first_affiliate_tracked

id                                  False
date_account_created                False
timestamp_first_active              False
date_first_booking                   True
gender                              False
age                                 False
signup_method                       False
signup_flow                         False
language                            False
affiliate_channel                   False
affiliate_provider                  False
first_affiliate_tracked              True
signup_app                          False
first_device_type                   False
first_browser                       False
country_destination                 False
lag_account_created                 False
lag_first_active                    False
lag_account_created_first_active    False
dtype: bool

In [24]:
population_in_thous = []
for i in range(train.shape[0]):
    if train.gender[i] == '-unknown-' or train.gender[i] == 'OTHER' or train.age[i] == 'NA' or \
    train.country_destination[i] == 'NDF' or train.country_destination[i] == 'other': 
        population_in_thous.append('NA')
    else:
        genders = user_demo.loc[user_demo.gender == train.gender[i].lower(),:] 

        dests = genders.loc[genders.country_destination == train.country_destination[i] ,:]    
        population_in_thous.append(float((dests.loc[dests.age_bucket == train.age[i], 'population_in_thousands'])))
        

In [25]:
train['population_in_thousands'] = population_in_thous

In [26]:
#merging with grouped sessions and countries, **note most of training data is not in sessions. see below 
test = pd.merge(test, group_sessions, left_on='id', right_on ='user_id', how='left')
train = pd.merge(train, group_sessions, left_on='id', right_on ='user_id', how='left')
test = test.drop('user_id', 1)
train = train.drop('user_id', 1)
train_m = pd.merge(train, countries, on='country_destination', how='left')

In [27]:
train_m = train_m.rename(columns={'destination_language ': 'destination_language'})

In [28]:
train_m.columns

Index([u'id', u'date_account_created', u'timestamp_first_active',
       u'date_first_booking', u'gender', u'age', u'signup_method',
       u'signup_flow', u'language', u'affiliate_channel',
       u'affiliate_provider', u'first_affiliate_tracked', u'signup_app',
       u'first_device_type', u'first_browser', u'country_destination',
       u'lag_account_created', u'lag_first_active',
       u'lag_account_created_first_active', u'population_in_thousands',
       u'sum_secs_elapsed', u'counts', u'lat_destination', u'lng_destination',
       u'distance_km', u'destination_km2', u'destination_language',
       u'language_levenshtein_distance'],
      dtype='object')

In [29]:
#remove id, time format columns and categorical columns
excl = list(train_m.columns)
toremove = ['id', 'date_account_created', 'timestamp_first_active', 'date_first_booking', 
            'population_in_thousands', 'gender', 'age', 'country_destination',
            'signup_method', 'language', 'affiliate_channel', 'affiliate_provider', 
            'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser', 
            'lag_account_created', 'lag_first_active', 'lag_account_created_first_active', 'destination_language'] 
map(lambda x: excl.remove(x), toremove)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [30]:
train_x = train_m.loc[:, excl]

In [31]:
print pd.get_dummies(train_m.gender).head()
print pd.get_dummies(train_m.age).head()
print pd.get_dummies(train_m.signup_method).head()
print pd.get_dummies(train_m.language).head()
print pd.get_dummies(train_m.affiliate_channel).head()
print pd.get_dummies(train_m.affiliate_provider).head()
print pd.get_dummies(train_m.first_affiliate_tracked).head()
print pd.get_dummies(train_m.signup_app).head()
print pd.get_dummies(train_m.first_device_type).head()
print pd.get_dummies(train_m.first_browser).head()
print pd.get_dummies(train_m.country_destination).head()
print pd.get_dummies(train_m.lag_account_created).head()
print pd.get_dummies(train_m.lag_first_active).head()
print pd.get_dummies(train_m.lag_account_created_first_active).head()
print pd.get_dummies(train_m.destination_language).head()

   -unknown-  FEMALE  MALE  OTHER
0        1.0     0.0   0.0    0.0
1        0.0     0.0   1.0    0.0
2        0.0     1.0   0.0    0.0
3        0.0     1.0   0.0    0.0
4        1.0     0.0   0.0    0.0
   0-4  100+  15-19  20-24  25-29  30-34  35-39  40-44  45-49  5-9 ...   \
0  0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  0.0 ...    
1  0.0   0.0    0.0    0.0    0.0    0.0    1.0    0.0    0.0  0.0 ...    
2  0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  0.0 ...    
3  0.0   0.0    0.0    0.0    0.0    0.0    0.0    1.0    0.0  0.0 ...    
4  0.0   0.0    0.0    0.0    0.0    0.0    0.0    1.0    0.0  0.0 ...    

   55-59  60-64  65-69  70-74  75-79  80-84  85-89  90-94  95-99   NA  
0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  1.0  
1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  0.0  
2    1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  0.0  
3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0

In [32]:
#gender
train_x[list(pd.get_dummies(train_m.gender).columns)[0]] = pd.get_dummies(train_m.gender).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.gender).columns))):
    train_x[list(pd.get_dummies(train_m.gender).columns)[i]] = pd.get_dummies(train_m.gender).iloc[:,i]
train_x['gender_unknown'] = pd.get_dummies(train_m.gender).iloc[:,0]
train_x['gender_other'] = pd.get_dummies(train_m.gender).iloc[:,3]

In [33]:

#age
train_x[list(pd.get_dummies(train_m.age).columns)[0]] = pd.get_dummies(train_m.age).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.age).columns))):
    train_x[list(pd.get_dummies(train_m.age).columns)[i]] = pd.get_dummies(train_m.age).iloc[:,i]
train_x['age_NA'] = pd.get_dummies(train_m.age).iloc[:,20]

#signup_method
train_x[list(pd.get_dummies(train_m.signup_method).columns)[0]] = pd.get_dummies(train_m.signup_method).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.signup_method).columns))):
    train_x[list(pd.get_dummies(train_m.signup_method).columns)[i]] = pd.get_dummies(train_m.signup_method).iloc[:,i]

#language
train_x[list(pd.get_dummies(train_m.language).columns)[0]] = pd.get_dummies(train_m.language).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.language).columns))):
    train_x[list(pd.get_dummies(train_m.language).columns)[i]] = pd.get_dummies(train_m.language).iloc[:,i]

#affiliate_channel
train_x[list(pd.get_dummies(train_m.affiliate_channel).columns)[0]] = pd.get_dummies(train_m.affiliate_channel).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.affiliate_channel).columns))):
    train_x[list(pd.get_dummies(train_m.affiliate_channel).columns)[i]] = pd.get_dummies(train_m.affiliate_channel).iloc[:,i]

#affiliate_provider
train_x[list(pd.get_dummies(train_m.affiliate_provider).columns)[0]] = pd.get_dummies(train_m.affiliate_provider).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.affiliate_provider).columns))):
    train_x[list(pd.get_dummies(train_m.affiliate_provider).columns)[i]] = pd.get_dummies(train_m.affiliate_provider).iloc[:,i]

#first_affiliate_tracked
train_x[list(pd.get_dummies(train_m.first_affiliate_tracked).columns)[0]] = pd.get_dummies(train_m.first_affiliate_tracked).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.first_affiliate_tracked).columns))):
    train_x[list(pd.get_dummies(train_m.first_affiliate_tracked).columns)[i]] = pd.get_dummies(train_m.first_affiliate_tracked).iloc[:,i]

#signup_app
train_x[list(pd.get_dummies(train_m.signup_app).columns)[0]] = pd.get_dummies(train_m.signup_app).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.signup_app).columns))):
    train_x[list(pd.get_dummies(train_m.signup_app).columns)[i]] = pd.get_dummies(train_m.signup_app).iloc[:,i]

#first_device_type
train_x[list(pd.get_dummies(train_m.first_device_type).columns)[0]] = pd.get_dummies(train_m.first_device_type).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.first_device_type).columns))):
    train_x[list(pd.get_dummies(train_m.first_device_type).columns)[i]] = pd.get_dummies(train_m.first_device_type).iloc[:,i]

#first_browser
train_x[list(pd.get_dummies(train_m.first_browser).columns)[0]] = pd.get_dummies(train_m.first_browser).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.first_browser).columns))):
    train_x[list(pd.get_dummies(train_m.first_browser).columns)[i]] = pd.get_dummies(train_m.first_browser).iloc[:,i]
train_x['first_browser_unknown'] = pd.get_dummies(train_m.first_browser).iloc[:,0]
    
#country_destination
train_x[list(pd.get_dummies(train_m.country_destination).columns)[0]] = pd.get_dummies(train_m.country_destination).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.country_destination).columns))):
    train_x[list(pd.get_dummies(train_m.country_destination).columns)[i]] = pd.get_dummies(train_m.country_destination).iloc[:,i]
train_x['country_dest_other'] = pd.get_dummies(train_m.country_destination).iloc[:,11]

#lag_account_created
train_x['lag_account_created_NA'] = pd.get_dummies(train_m.lag_account_created).iloc[:,0]
train_x['lag_account_created_before'] = pd.get_dummies(train_m.lag_account_created).iloc[:,1]
train_x['lag_account_created_greater1'] = pd.get_dummies(train_m.lag_account_created).iloc[:,2]
train_x['lag_account_created_sameday'] = pd.get_dummies(train_m.lag_account_created).iloc[:,2]

#lag_first_active
train_x['lag_first_active_NA'] = pd.get_dummies(train_m.lag_first_active).iloc[:,0]
train_x['lag_first_active_before'] = pd.get_dummies(train_m.lag_first_active).iloc[:,1]
train_x['lag_first_active_greater1'] = pd.get_dummies(train_m.lag_first_active).iloc[:,2]
train_x['lag_first_active_sameday'] = pd.get_dummies(train_m.lag_first_active).iloc[:,2]

#lag_account_created_first_active
train_x['lag_account_created_first_active_before'] = pd.get_dummies(train_m.lag_account_created_first_active).iloc[:,0]
train_x['lag_account_created_first_active_greater1'] = pd.get_dummies(train_m.lag_account_created_first_active).iloc[:,1]
train_x['lag_account_created_first_active_sameday'] = pd.get_dummies(train_m.lag_account_created_first_active).iloc[:,2]

#destination_language
train_x[list(pd.get_dummies(train_m.destination_language).columns)[0]] = pd.get_dummies(train_m.destination_language).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.destination_language).columns))):
    train_x[list(pd.get_dummies(train_m.destination_language).columns)[i]] = pd.get_dummies(train_m.destination_language).iloc[:,i]

train_x = train_x.drop('-unknown-', 1)
train_x = train_x.drop('OTHER', 1)
train_x = train_x.drop('NA', 1)


In [34]:
pd.get_dummies(train_m.destination_language).columns

Index([u'deu', u'eng', u'fra', u'ita', u'nld', u'por', u'spa'], dtype='object')

In [35]:
list(train_x.columns)

['signup_flow',
 'sum_secs_elapsed',
 'counts',
 'lat_destination',
 'lng_destination',
 'distance_km',
 'destination_km2',
 'language_levenshtein_distance',
 'FEMALE',
 'MALE',
 'gender_unknown',
 'gender_other',
 '0-4',
 '100+',
 '15-19',
 '20-24',
 '25-29',
 '30-34',
 '35-39',
 '40-44',
 '45-49',
 '5-9',
 '50-54',
 '55-59',
 '60-64',
 '65-69',
 '70-74',
 '75-79',
 '80-84',
 '85-89',
 '90-94',
 '95-99',
 'age_NA',
 'basic',
 'facebook',
 'google',
 'ca',
 'cs',
 'da',
 'de',
 'el',
 'en',
 'es',
 'fi',
 'fr',
 'hr',
 'hu',
 'id',
 'is',
 'it',
 'ja',
 'ko',
 'nl',
 'no',
 'pl',
 'pt',
 'ru',
 'sv',
 'th',
 'tr',
 'zh',
 'api',
 'content',
 'direct',
 'other',
 'remarketing',
 'sem-brand',
 'sem-non-brand',
 'seo',
 'baidu',
 'bing',
 'craigslist',
 'daum',
 'email-marketing',
 'facebook-open-graph',
 'gsp',
 'meetup',
 'naver',
 'padmapper',
 'vast',
 'wayn',
 'yahoo',
 'yandex',
 'linked',
 'local ops',
 'marketing',
 'omg',
 'product',
 'tracked-other',
 'untracked',
 'Android',
 '

In [36]:
train_x.head()


,signup_flow,sum_secs_elapsed,counts,lat_destination,lng_destination,distance_km,destination_km2,language_levenshtein_distance,FEMALE,MALE,...,lag_account_created_first_active_before,lag_account_created_first_active_greater1,lag_account_created_first_active_sameday,deu,eng,fra,ita,nld,por,spa
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,NaN,NaN,36.966427,-95.84403,0.0,9826675.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,NaN,NaN,36.966427,-95.84403,0.0,9826675.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [37]:
train_y = train_m.loc[:,'country_destination']

In [38]:
titanic = train_x.fillna(-999)
#pd.isnull(titanic).any()
titanic.head()

,signup_flow,sum_secs_elapsed,counts,lat_destination,lng_destination,distance_km,destination_km2,language_levenshtein_distance,FEMALE,MALE,...,lag_account_created_first_active_before,lag_account_created_first_active_greater1,lag_account_created_first_active_sameday,deu,eng,fra,ita,nld,por,spa
0,0,-999.0,-999.0,-999.000000,-999.00000,-999.0,-999.0,-999.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,-999.0,-999.0,-999.000000,-999.00000,-999.0,-999.0,-999.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,-999.0,-999.0,36.966427,-95.84403,0.0,9826675.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0,-999.0,-999.0,-999.000000,-999.00000,-999.0,-999.0,-999.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,-999.0,-999.0,36.966427,-95.84403,0.0,9826675.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [39]:
type(titanic) #DF
titanic.values

array([[  0.00000000e+00,  -9.99000000e+02,  -9.99000000e+02, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,  -9.99000000e+02,  -9.99000000e+02, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  3.00000000e+00,  -9.99000000e+02,  -9.99000000e+02, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [  0.00000000e+00,   3.42982000e+05,   1.80000000e+01, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.50000000e+01,   3.41609000e+05,   7.50000000e+01, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.50000000e+01,   2.75921000e+06,   4.10000000e+01, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [40]:
titanic_new = np.hstack((titanic))
type(titanic_new)

numpy.ndarray

In [41]:
titanic_new = np.hstack((titanic))
type(titanic_new)
titanic_new

array(['signup_flow', 'sum_secs_elapsed', 'counts', 'lat_destination',
       'lng_destination', 'distance_km', 'destination_km2',
       'language_levenshtein_distance', 'FEMALE', 'MALE', 'gender_unknown',
       'gender_other', '0-4', '100+', '15-19', '20-24', '25-29', '30-34',
       '35-39', '40-44', '45-49', '5-9', '50-54', '55-59', '60-64',
       '65-69', '70-74', '75-79', '80-84', '85-89', '90-94', '95-99',
       'age_NA', 'basic', 'facebook', 'google', 'ca', 'cs', 'da', 'de',
       'el', 'en', 'es', 'fi', 'fr', 'hr', 'hu', 'id', 'is', 'it', 'ja',
       'ko', 'nl', 'no', 'pl', 'pt', 'ru', 'sv', 'th', 'tr', 'zh', 'api',
       'content', 'direct', 'other', 'remarketing', 'sem-brand',
       'sem-non-brand', 'seo', 'baidu', 'bing', 'craigslist', 'daum',
       'email-marketing', 'facebook-open-graph', 'gsp', 'meetup', 'naver',
       'padmapper', 'vast', 'wayn', 'yahoo', 'yandex', 'linked',
       'local ops', 'marketing', 'omg', 'product', 'tracked-other',
       'untracked',

In [42]:
titanic_new[0].size

1

In [43]:
train.rename(columns={'country_destination': 'class'}, inplace=True)


In [44]:
# train = pd.read_csv('data/train_users_2.csv')
# train.rename(columns={'country_destination': 'class'}, inplace=True)
titanic_class = train['class'].values


In [45]:
train['class'].unique()

array(['NDF', 'US', 'other', 'FR', 'CA', 'GB', 'ES', 'IT', 'PT', 'NL',
       'DE', 'AU'], dtype=object)

In [46]:
train_labels = train['class'].values
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_labels)
train_labels = le.transform(train_labels)
print train_labels
le.inverse_transform(train_labels)

[ 7  7 10 ...,  7  7  7]


array(['NDF', 'NDF', 'US', ..., 'NDF', 'NDF', 'NDF'], dtype=object)

In [ ]:
#le = preprocessing.LabelEncoder()

In [ ]:
tpot = TPOTClassifier(generations=2, verbosity=2)
tpot.fit(titanic.values, train_labels)

GP Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]